In [1]:
suppressMessages(library(monocle))
suppressMessages(library(dplyr))
suppressMessages(library(tidyverse))

Warning message:
“package ‘monocle’ was built under R version 4.3.2”
Warning message:
“package ‘Matrix’ was built under R version 4.3.2”
Warning message:
“package ‘Biobase’ was built under R version 4.3.2”
Warning message:
“package ‘BiocGenerics’ was built under R version 4.3.2”


In [3]:
input_folder = "path/to/cell/metadata"
df_cell <- read.csv(file.path(input_folder, "df_cell_PanSci.csv"))

In [4]:
df_cell$sub_cell_type_indi_ID = str_c(df_cell$Conditions, 
                                      df_cell$subcluster_organ, sep = ".")
df_cell_count_sub_cell_type = df_cell %>% group_by(sub_cell_type_indi_ID, genotype, age_group, gender, project, ID, organ_ID, subcluster_organ) %>% summarise(cell_num = n())

`summarise()` has grouped output by 'sub_cell_type_indi_ID', 'genotype',
'age_group', 'gender', 'project', 'ID', 'organ_ID'. You can override using the
`.groups` argument.


In [5]:
head(df_cell_count_sub_cell_type)

sub_cell_type_indi_ID,genotype,age_group,gender,project,ID,organ_ID,subcluster_organ,cell_num
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>
1-Aging-23_months-Male-WT.Adipocytes-0-Liver,WT,23_months,Male,Aging,1,04_Liver,Adipocytes-0-Liver,22
1-Aging-23_months-Male-WT.Adipocytes-1-Liver,WT,23_months,Male,Aging,1,04_Liver,Adipocytes-1-Liver,8
1-Aging-23_months-Male-WT.Adipocytes-2-Liver,WT,23_months,Male,Aging,1,04_Liver,Adipocytes-2-Liver,1
1-Aging-23_months-Male-WT.Adipocytes-3-Liver,WT,23_months,Male,Aging,1,04_Liver,Adipocytes-3-Liver,1
1-Aging-23_months-Male-WT.Adipocytes-4-Liver,WT,23_months,Male,Aging,1,04_Liver,Adipocytes-4-Liver,28
1-Aging-23_months-Male-WT.Cacna1b positive cells-4-Liver,WT,23_months,Male,Aging,1,04_Liver,Cacna1b positive cells-4-Liver,1


In [6]:
organ_names = unique(df_cell_count_sub_cell_type$organ_ID)

In [11]:
# Calculate the proportion of each cell type in each sample
cds_cell_count_proportion_matrix <- function(cds) {
    b = Matrix::Diagonal(x = 1 / colSums(exprs(cds)))
    return(exprs(cds) %*% b)
}

# Calculate the proportion of each cell type in each sample
cds_cell_count_class_mean <- function(cds, class_vector) {
    cds_express <- (cds_cell_count_proportion_matrix(cds))
    unique_class <- sort(unique(class_vector))
    tmp_class_means <- lapply(1:length(unique_class), 
                            function(x) {
                                if(sum(class_vector == unique_class[x]) > 1) {
                                    result = Matrix::rowMeans(cds_express[, class_vector == unique_class[x]]) }
                                else {
                                result = cds_express[, class_vector == unique_class[x]] }
                                return(result) })
    
    tmp_class_means <- do.call(cbind, tmp_class_means)
    colnames(tmp_class_means) = unique_class
    return(tmp_class_means)
}

## Construct cds object
cds_construct <- function (UMI, df_cell, df_gene) 
{
    df_cell = as.data.frame(df_cell)
    df_gene = as.data.frame(df_gene)
    df_gene = df_gene %>% plyr::rename(c(gene_name = "gene_short_name"))
    pd = new("AnnotatedDataFrame", data = df_cell)
    fd = new("AnnotatedDataFrame", data = df_gene)
    colnames(UMI) = df_cell$sample
    row.names(UMI) = df_gene$gene_id
    row.names(pd) = colnames(UMI)
    row.names(fd) = row.names(UMI)
    cds = newCellDataSet(UMI, phenoData = pd, featureData = fd, 
        expressionFamily = negbinomial.size())
    return(cds)
}

In [12]:
cds_sub_cell_type_list = list()
df_cell_proportion_age_list = list()

In [13]:
for(organ_name in organ_names) {
    cat("\nProcessing sample: ")
    cat(organ_name)
    df_organ_cell = df_cell_count_sub_cell_type %>% filter(organ_ID == organ_name)

    df_organ_cell$sample = str_split_fixed(df_organ_cell$sub_cell_type_indi_ID, pattern = "\\.", n = 2)[, 1]

    df_cell_count = as.data.frame(df_organ_cell) %>% select(sample, subcluster_organ, cell_num) %>% spread(key = subcluster_organ, value = cell_num, fill = 0)

    df_cell_mm = as.matrix(df_cell_count %>% select(-sample))
    rownames(df_cell_mm) = df_cell_count$sample

    df_cell_mm = t(df_cell_mm)

    df_cell_mm_anno = data.frame(sample = colnames(df_cell_mm))
    df_tmp = as.data.frame(df_organ_cell) %>% select(sample, genotype, age_group, gender, ID, organ_ID) %>% unique()
    df_cell_mm_anno = left_join(df_cell_mm_anno, df_tmp)

    df_cell_mm_cell_type_anno = data.frame(gene_id = rownames(df_cell_mm), gene_name = rownames(df_cell_mm), subcluster_organ = rownames(df_cell_mm))
    df_cell_mm_cell_type_anno$sub_cell_type = str_split_fixed(df_cell_mm_cell_type_anno$subcluster_organ, pattern = "-", n = 2)[, 1]
    df_cell_mm_cell_type_anno$organ = str_split_fixed(df_cell_mm_cell_type_anno$subcluster_organ, pattern = "-", n = 2)[, 2]

    cds_cell_count = cds_construct(df_cell_mm, df_cell_mm_anno, df_cell_mm_cell_type_anno)

    cds_cell_count = estimateSizeFactors(cds_cell_count)
    cds_cell_count$age_genotype = str_c(cds_cell_count$age_group, cds_cell_count$genotype, sep = "-")
    
    df_cell_proportion = cds_cell_count_proportion_matrix(cds_cell_count)

    df_cell_proportion_age_genotype = cds_cell_count_class_mean(cds_cell_count, cds_cell_count$age_genotype)

    # Identify cds object that are significantly changed between 06 and 12 months/03 and 16 months
    cds_sampled = cds_cell_count[, cds_cell_count$age_group %in% c("06_months", "23_months")]
    DA_cells = differentialGeneTest(cds_sampled, fullModelFormulaStr = "~ Size_Factor + age_group", 
            reducedModelFormulaStr = "~ Size_Factor")
    DA_cells = as.data.frame(DA_cells) %>% select(pval_06_23 = pval, qval_06_23 = qval, cell_type_id = gene_id)

    df_cell_proportion_age_genotype = as.data.frame(df_cell_proportion_age_genotype)
    df_cell_proportion_age_genotype$cell_type_id = rownames(df_cell_proportion_age_genotype) 
    df_cell_proportion_age_genotype = left_join(df_cell_proportion_age_genotype, DA_cells)

    cds_sampled = cds_cell_count[, (cds_cell_count$age_group %in% c("03_months", "16_months")) & (cds_cell_count$genotype == "WT")]
    DA_cells = differentialGeneTest(cds_sampled, fullModelFormulaStr = "~ Size_Factor + age_group", 
            reducedModelFormulaStr = "~ Size_Factor")
    DA_cells = as.data.frame(DA_cells) %>% select(pval_03_16 = pval, qval_03_16 = qval, cell_type_id = gene_id)
    df_cell_proportion_age_genotype = left_join(df_cell_proportion_age_genotype, DA_cells)
    
    cds_sub_cell_type_list[[organ_name]] = cds_cell_count
    df_cell_proportion_age_list[[organ_name]] = df_cell_proportion_age_genotype
    }


Processing sample: 04_Liver

Joining with `by = join_by(sample)`
Joining with `by = join_by(cell_type_id)`
Joining with `by = join_by(cell_type_id)`



Processing sample: 07_BAT

Joining with `by = join_by(sample)`
Joining with `by = join_by(cell_type_id)`
Joining with `by = join_by(cell_type_id)`



Processing sample: 09_gWAT

...

Joining with `by = join_by(sample)`
Joining with `by = join_by(cell_type_id)`
Joining with `by = join_by(cell_type_id)`



Processing sample: 11_Colon

Joining with `by = join_by(sample)`
Joining with `by = join_by(cell_type_id)`
Joining with `by = join_by(cell_type_id)`



Processing sample: 10_Ileum

Joining with `by = join_by(sample)`
Joining with `by = join_by(cell_type_id)`
Joining with `by = join_by(cell_type_id)`


In [14]:
df_cell_proportion_age = bind_rows(df_cell_proportion_age_list)

In [15]:
df_cell_proportion_age$qval_06_23_corrected = p.adjust(df_cell_proportion_age$pval_06_23, method = "fdr")
df_cell_proportion_age$qval_03_16_corrected = p.adjust(df_cell_proportion_age$pval_03_16, method = "fdr")

df_cell_proportion_age$LFC_23_over_06 = log2((df_cell_proportion_age$`23_months-WT` + 1e-6) / (df_cell_proportion_age$`06_months-WT`+ 1e-6))
df_cell_proportion_age$LFC_16_over_03 = log2((df_cell_proportion_age$`16_months-WT` + 1e-6) / (df_cell_proportion_age$`03_months-WT`+ 1e-6))